In [3]:
from jurbey.jurbey import JURBEY
from sklearn.decomposition import NMF
import networkx as nx
import numpy as np

f = open('1558537930325.jurbey','rb')
g = JURBEY.load(f.read())

for e in g.edges(data=True):
    try:
        g[e[0]][e[1]]["speed"] = float(g[e[0]][e[1]]["data"].metadata.get("maxspeed",10))
    except ValueError:
        g[e[0]][e[1]]["speed"] = 10

In [4]:
A = nx.adjacency_matrix(g,weight="speed")
node_list = list(g.nodes())
edge_list = list(g.edges())

In [5]:
from sklearn.preprocessing import MaxAbsScaler
transformer = MaxAbsScaler().fit(A)
A_norm = transformer.transform(A)

In [ ]:
model_mu = NMF(n_components=50, init='random', random_state=0, verbose=True, solver='mu', tol=0.000001)
W_mu = model.fit_transform(A_norm)

In [ ]:
model_cd = NMF(n_components=50, init='nndsvda', random_state=0, verbose=True, solver='cd', tol=0.0001)
W_cd = model_cd.fit_transform(A_norm)

In [ ]:
actual = A_norm[node_list.index(edge_list[1000][0]), node_list.index(edge_list[1000][1])]
pred_mu = np.dot(W_mu[node_list.index(edge_list[1000][0])],model.components_.T[node_list.index(edge_list[1000][1])])
pred_cd = np.dot(W_cd[node_list.index(edge_list[1000][0])],model.components_.T[node_list.index(edge_list[1000][1])])

print(f'actual: {actual} and pred mu: {pred_mu} and pred cd: {pred_cd}')

In [131]:
def mf(X, K, iterations=1000, alpha=0.0002, beta=0.02):
    print(X.shape)
    m,n = X.shape
    W = np.random.rand(m,K)
    H = np.random.rand(n,K)
    H = H.T
    for _iter in range(iterations):
        for i in range(m):
            for j in range(n):
                if X[i, j] > 0 and not np.isnan(X[i, j]):
                    eij = X[i,j] - np.dot(W[i, :], H[:, j])
                    for k in range(K):
                        # multiplicative update + L2 regularization
                        W[i][k] = W[i][k] + alpha * (2 * eij * H[k][j] - beta * W[i][k])
                        H[k][j] = H[k][j] + alpha * (2 * eij * W[i][k] - beta * H[k][j])
        tol_list = list()                
        tol = 0
        for i in range(m):
            for j in range(n):
                if X[i, j] > 0 and not np.isnan(X[i, j]):
                    # Frobenius norm
                    tol = tol + pow(X[i,j] - np.dot(W[i, :], H[:, j]), 2)
                    for k in range(K):
                        tol = tol + (beta / 2) * (pow(W[i][k], 2) + pow(H[k][j], 2))
        print(f'At iter: {_iter}, tol: {tol}')
        tol_list.append(tol)
        if tol < 0.001:
            break
    return W, H.T, tol_list

In [59]:
import random
sampled_nodes = random.sample(g.nodes, 1000)
g_sampled = nx.DiGraph(g).subgraph(sampled_nodes)
A = nx.adjacency_matrix(g_sampled,weight="speed").todense()

In [47]:
print(np.shape(A))
print(A[0,1])

(10000, 10000)
0.0


In [74]:
W,H = mf(A, 10)

At iter: 0, tol: 2389.72508765189
At iter: 1, tol: 2376.865667901268
At iter: 2, tol: 2363.752580380483
At iter: 3, tol: 2350.3735405699103
At iter: 4, tol: 2336.716427445819
At iter: 5, tol: 2322.7692970119406
At iter: 6, tol: 2308.52039791039
At iter: 7, tol: 2293.958189138839
At iter: 8, tol: 2279.071359896901
At iter: 9, tol: 2263.848851579729
At iter: 10, tol: 2248.279881930264
At iter: 11, tol: 2232.3539713538353
At iter: 12, tol: 2216.0609713895838
At iter: 13, tol: 2199.3910953223267
At iter: 14, tol: 2182.334950906207
At iter: 15, tol: 2164.8835751574697
At iter: 16, tol: 2147.0284711580525
At iter: 17, tol: 2128.7616467944313
At iter: 18, tol: 2110.075655337111
At iter: 19, tol: 2090.9636377455286
At iter: 20, tol: 2071.4193665608927
At iter: 21, tol: 2051.4372912256185
At iter: 22, tol: 2031.0125846428377
At iter: 23, tol: 2010.1411907628726
At iter: 24, tol: 1988.8198729559253
At iter: 25, tol: 1967.0462629017006
At iter: 26, tol: 1944.8189096974572
At iter: 27, tol: 1922.1

KeyboardInterrupt: 

In [37]:
print(np.shape(W))

(10000, 10)


In [88]:
node_list = list(g_sampled.nodes())
edge_list = list(g_sampled.edges())
print(np.shape(W))
print(np.shape(H))
print(len(edge_list))
print(W[node_list.index(edge_list[1][0])])
print(H[node_list.index(edge_list[1][1])])
actual = A[node_list.index(edge_list[1][0]), node_list.index(edge_list[1][1])]
pred = np.dot(W[node_list.index(edge_list[1][0])],H[node_list.index(edge_list[1][1])])

print(f'actual: {actual} and pred: {pred}')

(1992, 10)
(1992, 10)
1859
[0.62672462 0.5190578  0.8640665  0.9233341  1.14229385 0.70034687
 0.90799164 0.7385996  0.37466045 0.99507168]
[0.94729473 0.55693807 0.29132277 0.40101646 0.96077029 0.31572068
 0.47013893 0.35128026 0.93315479 0.1294371 ]
actual: 0.0 and pred: 3.9881194364362234


In [112]:
import random
import itertools
sampled_edges = random.sample(g.edges, 100)
sampled_nodes = set(itertools.chain(*sampled_edges))
g_sampled = nx.DiGraph(g).subgraph(sampled_nodes)
node_list = list(g_sampled.nodes())
edge_list = list(g_sampled.edges())

A2 = nx.adjacency_matrix(g_sampled,weight="speed").todense()
from sklearn.preprocessing import MaxAbsScaler
transformer = MaxAbsScaler().fit(A2)
A2 = transformer.transform(A2)
gt_edges = random.sample(sampled_edges,10)
for u,v in gt_edges:
    g_sampled[u][v]["old_speed"] = A2[node_list.index(u), node_list.index(v)]
    A2[node_list.index(u), node_list.index(v)] = np.nan


In [118]:
W,H = mf(A2, 5, iterations=3000)

(200, 200)
At iter: 0, tol: 55.52954897911336
At iter: 1, tol: 55.358673129317275
At iter: 2, tol: 55.18852793868183
At iter: 3, tol: 55.019109407039885
At iter: 4, tol: 54.85041356193248
At iter: 5, tol: 54.682436458372464
At iter: 6, tol: 54.51517417861106
At iter: 7, tol: 54.34862283190441
At iter: 8, tol: 54.18277855428603
At iter: 9, tol: 54.01763750833796
At iter: 10, tol: 53.853195882966766
At iter: 11, tol: 53.689449893181276
At iter: 12, tol: 53.52639577987126
At iter: 13, tol: 53.36402980959048
At iter: 14, tol: 53.20234827434002
At iter: 15, tol: 53.04134749135506
At iter: 16, tol: 52.881023802893075
At iter: 17, tol: 52.721373576024526
At iter: 18, tol: 52.562393202425575
At iter: 19, tol: 52.404079098172694
At iter: 20, tol: 52.246427703539815
At iter: 21, tol: 52.089435482796645
At iter: 22, tol: 51.933098924010025
At iter: 23, tol: 51.77741453884621
At iter: 24, tol: 51.62237886237676
At iter: 25, tol: 51.46798845288312
At iter: 26, tol: 51.314239891667995
At iter: 27, t

In [119]:
node_list = list(g_sampled.nodes())
edge_list = list(g_sampled.edges())
y_actual = list()
y_pred = list()
for u,v in gt_edges:
    actual = g_sampled[u][v]["old_speed"]
    pred = np.dot(W[node_list.index(u)],H[node_list.index(v)])
    print(f'actual: {actual} and pred: {pred}')
    y_actual.append(actual)
    y_pred.append(pred)
    
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_actual, y_pred))
print(f'rmse: {rmse}')



    



actual: 1.0 and pred: 1.7742350201995636
actual: 1.0 and pred: 0.8871806663668976
actual: 1.0 and pred: 0.8053794983181931
actual: 1.0 and pred: 1.6362250804636276
actual: 1.0 and pred: 1.652635000087666
actual: 1.0 and pred: 1.5203613533913294
actual: 1.0 and pred: 0.9055475504083424
actual: 1.0 and pred: 0.8868557310323133
actual: 1.0 and pred: 1.602324014828519
actual: 1.0 and pred: 1.060788923572789
rmse: 0.46257413952530124


In [115]:
print(f'original density: {nx.density(g)} and sampled graph density: {nx.density(g_sampled)}')

original density: 6.615388352506922e-06 and sampled graph density: 0.004623115577889447


In [126]:
sampled_nodes = random.sample(g.nodes, 5000)
g_sampled = nx.DiGraph(g).subgraph(sampled_nodes)
print(g_sampled.number_of_edges())
node_list = list(g_sampled.nodes())
edge_list = list(g_sampled.edges())

160


In [130]:
A3 = nx.adjacency_matrix(g_sampled,weight="speed").todense()
from sklearn.preprocessing import MaxAbsScaler
transformer = MaxAbsScaler().fit(A3)
A3 = transformer.transform(A3)
gt_edges = random.sample(g_sampled.edges(),10)
for u,v in gt_edges:
    g_sampled[u][v]["old_speed"] = A3[node_list.index(u), node_list.index(v)]
    A3[node_list.index(u), node_list.index(v)] = np.nan

In [134]:
W,H, tol_list = mf(A3, 5, iterations=100)

(5000, 5000)
At iter: 0, tol: 47.41714849064951
At iter: 1, tol: 47.26548822551438
At iter: 2, tol: 47.11451232061051
At iter: 3, tol: 46.96421673037875
At iter: 4, tol: 46.814597440247695
At iter: 5, tol: 46.66565046633589
At iter: 6, tol: 46.51737185515733
At iter: 7, tol: 46.36975768332952
At iter: 8, tol: 46.22280405728684
At iter: 9, tol: 46.07650711299513
At iter: 10, tol: 45.93086301567102
At iter: 11, tol: 45.78586795950377
At iter: 12, tol: 45.64151816738071
At iter: 13, tol: 45.497809890615294
At iter: 14, tol: 45.354739408679414
At iter: 15, tol: 45.21230302893719
At iter: 16, tol: 45.07049708638339
At iter: 17, tol: 44.92931794338322
At iter: 18, tol: 44.78876198941676
At iter: 19, tol: 44.64882564082506
At iter: 20, tol: 44.50950534055974
At iter: 21, tol: 44.370797557934765
At iter: 22, tol: 44.23269878838203
At iter: 23, tol: 44.095205553208835
At iter: 24, tol: 43.95831439935837
At iter: 25, tol: 43.82202189917326
At iter: 26, tol: 43.68632465016079
At iter: 27, tol: 43

In [135]:
node_list = list(g_sampled.nodes())
edge_list = list(g_sampled.edges())
y_actual = list()
y_pred = list()
for u,v in gt_edges:
    actual = g_sampled[u][v]["old_speed"]
    pred = np.dot(W[node_list.index(u)],H[node_list.index(v)])
    print(f'actual: {actual} and pred: {pred}')
    y_actual.append(actual)
    y_pred.append(pred)
    
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_actual, y_pred))
print(f'rmse: {rmse}')

actual: 1.0 and pred: 1.3434074713232427
actual: 1.0 and pred: 1.5216560647407107
actual: 1.0 and pred: 1.5071931219612975
actual: 1.0 and pred: 0.6991363375232821
actual: 1.0 and pred: 1.527460120498063
actual: 1.0 and pred: 1.7877637918009377
actual: 1.0 and pred: 1.8452102294710289
actual: 1.0 and pred: 1.1085486442451287
actual: 1.0 and pred: 1.7121922599152852
actual: 1.0 and pred: 1.8706156641394895
rmse: 0.6023251703101716


In [136]:
print(f'original density: {nx.density(g)} and sampled graph density: {nx.density(g_sampled)}')

original density: 6.615388352506922e-06 and sampled graph density: 6.40128025605121e-06
